# Developing Logistic Regression, kNN,GBTClassifier, AdaBoostClassifier, RandomForestClassifier models

## STEP 1 Installing libraries

In [22]:
# Importing the libraries
import numpy as np 
import matplotlib.pyplot as plt
import pandas as pd
# Importing the database
dataset = pd.read_csv('nhiesincml.csv',index_col = 0)

## STEP 2 Verifying, features

In [23]:
dataset.head()

,sex_of_head,age_group,age_of_head,reg_urb,main_language,main_sourceincome,tenure,urbrur_head,attain,apci,label_apci,ncons_exp,g01_food,income,cons_exp,apci_groups
0,2,3,35,2,5,1,3,4,3.0,23453.849609,3,1800.0,6681.848633,23453.849609,21653.849609,3
1,2,2,25,2,10,5,4,4,3.0,29355.007812,3,1800.0,10505.010742,29355.007812,27555.007812,3
2,1,4,51,2,5,1,1,3,2.0,91796.781250,4,2500.0,35028.773438,91796.781250,89296.773438,5
3,2,4,54,2,5,5,1,4,3.0,16954.039062,2,0.0,1756.040039,16954.039062,16954.039062,3
4,1,3,35,2,12,1,3,3,3.0,24646.001953,3,960.0,9542.000000,49292.003906,48332.000000,3


In [24]:
dataset.shape

(10090, 16)

In [25]:
dataset = dataset.drop(['reg_urb'], axis=1)

In [26]:
# convert attain to int
cols = ['attain','label_apci']
dataset[cols] = dataset[cols].applymap(np.int64)
dataset.head()

,sex_of_head,age_group,age_of_head,main_language,main_sourceincome,tenure,urbrur_head,attain,apci,label_apci,ncons_exp,g01_food,income,cons_exp,apci_groups
0,2,3,35,5,1,3,4,3,23453.849609,3,1800.0,6681.848633,23453.849609,21653.849609,3
1,2,2,25,10,5,4,4,3,29355.007812,3,1800.0,10505.010742,29355.007812,27555.007812,3
2,1,4,51,5,1,1,3,2,91796.781250,4,2500.0,35028.773438,91796.781250,89296.773438,5
3,2,4,54,5,5,1,4,3,16954.039062,2,0.0,1756.040039,16954.039062,16954.039062,3
4,1,3,35,12,1,3,3,3,24646.001953,3,960.0,9542.000000,49292.003906,48332.000000,3


In [27]:
# move sex_of_head col after age_of_head
cols = list(dataset)
cols.insert(2, cols.pop(cols.index('sex_of_head')))
dataset = dataset[cols]
dataset.head()

,age_group,age_of_head,sex_of_head,main_language,main_sourceincome,tenure,urbrur_head,attain,apci,label_apci,ncons_exp,g01_food,income,cons_exp,apci_groups
0,3,35,2,5,1,3,4,3,23453.849609,3,1800.0,6681.848633,23453.849609,21653.849609,3
1,2,25,2,10,5,4,4,3,29355.007812,3,1800.0,10505.010742,29355.007812,27555.007812,3
2,4,51,1,5,1,1,3,2,91796.781250,4,2500.0,35028.773438,91796.781250,89296.773438,5
3,4,54,2,5,5,1,4,3,16954.039062,2,0.0,1756.040039,16954.039062,16954.039062,3
4,3,35,1,12,1,3,3,3,24646.001953,3,960.0,9542.000000,49292.003906,48332.000000,3


## STEP 3 Engineering encoding features

In [28]:
X = dataset.iloc[:, 1:15].values
Y = dataset.iloc[:, 14].values
X

array([[3.50000000e+01, 2.00000000e+00, 5.00000000e+00, ...,
        2.34538496e+04, 2.16538496e+04, 3.00000000e+00],
       [2.50000000e+01, 2.00000000e+00, 1.00000000e+01, ...,
        2.93550078e+04, 2.75550078e+04, 3.00000000e+00],
       [5.10000000e+01, 1.00000000e+00, 5.00000000e+00, ...,
        9.17967812e+04, 8.92967734e+04, 5.00000000e+00],
       ...,
       [5.50000000e+01, 2.00000000e+00, 2.00000000e+00, ...,
        5.50095938e+04, 5.48695938e+04, 3.00000000e+00],
       [7.30000000e+01, 2.00000000e+00, 2.00000000e+00, ...,
        7.08872734e+04, 7.05372812e+04, 4.00000000e+00],
       [7.50000000e+01, 1.00000000e+00, 2.00000000e+00, ...,
        2.62423320e+04, 2.59923340e+04, 3.00000000e+00]])

In [29]:
Y

array([3, 3, 5, ..., 3, 4, 3])

In [30]:
ds = pd.DataFrame({'AgeHead':X[:,0],'SexHead':X[:,1],'Mainlang':X[:,2],'Mainsinc':X[:,3],\
                   'Tenure':X[:,4],'Urbrur':X[:,5],'Attain':X[:,6],'apci_groups':X[:,13]})

In [31]:
from sklearn.preprocessing import OneHotEncoder

SexHead_he = OneHotEncoder()
Mainlang_he = OneHotEncoder()
Mainsinc_he = OneHotEncoder()
Tenure_he = OneHotEncoder()
Urbrur_he = OneHotEncoder()
Attain_he = OneHotEncoder()
# apci_groups_he = OneHotEncoder()
SexHeadn = SexHead_he.fit_transform(ds.SexHead.values.reshape(-1,1)).toarray()
Mainlangn = Mainlang_he.fit_transform(ds.Mainlang.values.reshape(-1,1)).toarray()
Mainsincn = Mainsinc_he.fit_transform(ds.Mainsinc.values.reshape(-1,1)).toarray()
Tenuren = Tenure_he.fit_transform(ds.Tenure.values.reshape(-1,1)).toarray()
Urbrurn = Urbrur_he.fit_transform(ds.Urbrur.values.reshape(-1,1)).toarray()
Attainn = Attain_he.fit_transform(ds.Attain.values.reshape(-1,1)).toarray()
# apci_groups = apci_groups_he.fit_transform(ds.apci_groups.values.reshape(-1,1)).toarray()


/home/jupyterlab/conda/lib/python3.6/site-packages/sklearn/preprocessing/_encoders.py:368: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case you used a LabelEncoder before this OneHotEncoder to convert the categories to integers, then you can now use the OneHotEncoder directly.
  warnings.warn(msg, FutureWarning)
/home/jupyterlab/conda/lib/python3.6/site-packages/sklearn/preprocessing/_encoders.py:368: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'"

## STEP 4 Encoding features

In [32]:
dfOneHot = pd.DataFrame(SexHeadn, columns = ["SexHead_"+str(int(i)) for i in range(SexHeadn.shape[1])])
ds = pd.concat([ds, dfOneHot], axis=1)
dfOneHot = pd.DataFrame(Mainlangn, columns = ["Mainlang_"+str(int(i)) for i in range(Mainlangn.shape[1])])
ds = pd.concat([ds, dfOneHot], axis=1)
dfOneHot = pd.DataFrame(Mainsincn, columns = ["Mainsinc_"+str(int(i)) for i in range(Mainsincn.shape[1])])
ds = pd.concat([ds, dfOneHot], axis=1)
dfOneHot = pd.DataFrame(Tenuren, columns = ["Tenure_"+str(int(i)) for i in range(Tenuren.shape[1])])
ds = pd.concat([ds, dfOneHot], axis=1)
dfOneHot = pd.DataFrame(Urbrurn, columns = ["Urbrur_"+str(int(i)) for i in range(Urbrurn.shape[1])])
ds = pd.concat([ds, dfOneHot], axis=1)
dfOneHot = pd.DataFrame(Attainn, columns = ["Attain_"+str(int(i)) for i in range(Attainn.shape[1])])
ds = pd.concat([ds, dfOneHot], axis=1)

In [33]:
ds.head()

,AgeHead,SexHead,Mainlang,Mainsinc,Tenure,Urbrur,Attain,apci_groups,SexHead_0,SexHead_1,...,Urbrur_0,Urbrur_1,Urbrur_2,Urbrur_3,Attain_0,Attain_1,Attain_2,Attain_3,Attain_4,Attain_5
0,35.0,2.0,5.0,1.0,3.0,4.0,3.0,3.0,0.0,1.0,...,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0
1,25.0,2.0,10.0,5.0,4.0,4.0,3.0,3.0,0.0,1.0,...,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0
2,51.0,1.0,5.0,1.0,1.0,3.0,2.0,5.0,1.0,0.0,...,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
3,54.0,2.0,5.0,5.0,1.0,4.0,3.0,3.0,0.0,1.0,...,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0
4,35.0,1.0,12.0,1.0,3.0,3.0,3.0,3.0,1.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0


In [34]:
ds = ds.drop(['SexHead', 'Mainlang','Mainsinc', 'Tenure','Urbrur', 'Attain'], axis=1)  #'AgeHead'

In [35]:
ds.head()

,AgeHead,apci_groups,SexHead_0,SexHead_1,Mainlang_0,Mainlang_1,Mainlang_2,Mainlang_3,Mainlang_4,Mainlang_5,...,Urbrur_0,Urbrur_1,Urbrur_2,Urbrur_3,Attain_0,Attain_1,Attain_2,Attain_3,Attain_4,Attain_5
0,35.0,3.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0
1,25.0,3.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0
2,51.0,5.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
3,54.0,3.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0
4,35.0,3.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0


In [111]:
ds.to_csv('firsthotlregr.csv')

In [112]:
ds1 = pd.read_csv('firsthotlregr.csv', index_col=0)

In [113]:
ds1 = ds1.drop(['AgeHead','Tenure_0','Tenure_1','Tenure_2','Tenure_3','Tenure_4','Urbrur_0','Urbrur_1',\
              'Urbrur_2','Urbrur_3','Attain_0', 'Attain_1','Attain_2', 'Attain_3', 'Attain_4','Attain_5'], axis=1)

In [ ]:
ds1.to_csv('nhieslg')

In [114]:
ds1.head()

,apci_groups,SexHead_0,SexHead_1,Mainlang_0,Mainlang_1,Mainlang_2,Mainlang_3,Mainlang_4,Mainlang_5,Mainlang_6,...,Mainlang_11,Mainlang_12,Mainsinc_0,Mainsinc_1,Mainsinc_2,Mainsinc_3,Mainsinc_4,Mainsinc_5,Mainsinc_6,Mainsinc_7
0,3.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,3.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
2,5.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,3.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
4,3.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [115]:
X = ds1
Y = ds1.iloc[:, 0].values

In [116]:
X

,apci_groups,SexHead_0,SexHead_1,Mainlang_0,Mainlang_1,Mainlang_2,Mainlang_3,Mainlang_4,Mainlang_5,Mainlang_6,...,Mainlang_11,Mainlang_12,Mainsinc_0,Mainsinc_1,Mainsinc_2,Mainsinc_3,Mainsinc_4,Mainsinc_5,Mainsinc_6,Mainsinc_7
0,3.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,3.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
2,5.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,3.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
4,3.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,3.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,2.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
7,5.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9,2.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [117]:
Y

array([3., 3., 5., ..., 3., 4., 3.])

In [118]:
pd.value_counts(Y)

3.0    2689
2.0    2230
4.0    1900
1.0    1853
5.0     708
6.0     412
7.0     298
dtype: int64

## STEP 5 kNN Model

In [119]:
from sklearn.model_selection import cross_val_score

In [120]:
from sklearn.neighbors import KNeighborsClassifier

In [121]:
# 10-fold cross validation with K=5 for KNN
knn = KNeighborsClassifier(n_neighbors=15)
scores= cross_val_score(knn, X,Y,cv=5,scoring='accuracy')
scores.mean()

0.9223005911058373

## STEP 6 Linear Regression Model

In [129]:
from sklearn.linear_model import LogisticRegression
# 10-fold cross-validation with logistic regression
logreg=LogisticRegression()
# logreg.fit(X_train,y_train)
scores = cross_val_score(logreg, X,Y,cv=10,scoring='accuracy')
scores.mean()

/home/jupyterlab/conda/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/home/jupyterlab/conda/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:460: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)
/home/jupyterlab/conda/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/home/jupyterlab/conda/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:460: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)
/home/jupyterlab/conda/lib/python3.6/site-packages/sklearn/linear_model/lo

0.7079919501577996

## STEP 7 GBTClassifier , AdaBoostClassifier, RandomForestClassifier models

In [130]:
## GBTClassifier

In [132]:
from sklearn.ensemble import GradientBoostingClassifier
clf = GradientBoostingClassifier(n_estimators=200, learning_rate=1.0,max_depth=1, random_state=0).fit(X_train, y_train)
scores = cross_val_score(clf, X, Y, cv=10)
scores.mean() 

0.9704662358173193

In [133]:
## AdaBoostClassifier
from sklearn.ensemble import AdaBoostClassifier
clf = AdaBoostClassifier(n_estimators=100)
scores = cross_val_score(clf, X, Y, cv=10)
scores.mean() 

0.7006939933944802

In [134]:
from sklearn.ensemble import RandomForestClassifier

rclf = RandomForestClassifier()  
scores = cross_val_score(rclf, X, Y, cv=10)
scores.mean() 

/home/jupyterlab/conda/lib/python3.6/site-packages/sklearn/ensemble/forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
/home/jupyterlab/conda/lib/python3.6/site-packages/sklearn/ensemble/forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
/home/jupyterlab/conda/lib/python3.6/site-packages/sklearn/ensemble/forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
/home/jupyterlab/conda/lib/python3.6/site-packages/sklearn/ensemble/forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
/home/jupyterlab/conda/lib/python3.6/site-pa

0.9723425856465523